In [ ]:
flex_title = "PORTFOLIO POSITION OPTIMIZATION"
flex_orientation = "rows"

In [ ]:
import datetime as dt
import pytz

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import ipywidgets

**Papermill parameters**

In [ ]:
CLIENT = "TEST_FR"

**Other parameters**

In [ ]:
S3_DATASET = "s3://noos-prod-neptune-services/Store/Datasets/"

## Graph

### Portfolio exposure

#### consumption

In [ ]:
conso_df = pd.read_parquet(S3_DATASET + "NOOS/consommation_tot_france_random_1k.parquet")

#### trades

In [ ]:
QUERY_PARAMS = {
    "asset": "PWRTE",
    "portfolio_name": CLIENT,
}

trades_df = nptc.filter_forward_contracts(**QUERY_PARAMS)

In [ ]:
trades_df = trades_df[trades_df.intraday_delivery == 'BASE']

In [ ]:
trades_volume_df = npta.get_volumes_and_costs_from_trades(trades_df)

#### products

In [ ]:
# Retrieve all forward fixings on latest date

products_df = nptc.retrieve_forward_fixings(
         asset="PWRTE",
         published_at=dt.datetime(2021, 6, 1),
         min_delivery_period=dt.timedelta(hours=8)
)

#### graph updates

In [ ]:
def update_graph_hedges():
    global HEDGES_VOLUMES
    fig.data[2].update(y = (trades_volume_df.total_volume + HEDGES_VOLUMES).values * 2)

#### graph definition

In [ ]:
#  Create figure


trace0 = go.Scatter(
    x = conso_df.index,
    y = conso_df.consumption * 2,
    name = "consumption",
    visible = True
)


trace1 = go.Scatter(
    x = trades_volume_df.index,
    y = trades_volume_df.total_volume * 2,
    name = "trades",
    fill='tozeroy',
    visible = True
)

trace2 = go.Scatter(
    x = trades_volume_df.index,
    y = [],
    name = "with hedges",
    mode='lines+markers',
    visible = True
)


fig = go.FigureWidget(
    data=[trace0, trace1, trace2],
    layout=dict(
#         title="Volume Exposition",
        xaxis_title="Delivery date",
        yaxis=dict(
            title="MWh",
            fixedrange = True,
        ),
        template='noos_watermark+noos_base+hoover_xunified+xaxis_period+min_modebar',
    )
)

In [ ]:
fig

## Summary

In [ ]:
def get_eex_curve(wholesale_date:dt.datetime):
    ts = nptc.retrieve_time_series(
    curve_type="PRICE",
    curve_uid="PWRTE",
    published_at=wholesale_date,
)
    
    return pd.DataFrame(data={'price': ts}, index=ts.index.rename('delivery_from'))

In [ ]:
%time curve_pwrte_df=get_eex_curve(dt.datetime(2021,5,31))

In [ ]:
summary_hedges_widget = ipywidgets.HTML()
summary_trading_widget = ipywidgets.HTML()
summary_shaping_widget = ipywidgets.HTML()

In [ ]:
def print_summary_trading(*, 
                          traded_ratio_before, 
                          traded_ratio_after, 
                          traded_volume_before,
                          traded_volume_after,
                          traded_cost_before, 
                          traded_cost_after, 
                          traded_priceavg_before,
                          traded_priceavg_after,
                         ):
    return f"""
    <table class="summary">
        <tbody>
            <tr>
                <td></td>
                <th>Actual</th>
                <th>Proposed</th>
            </tr>
            <tr>
                <th>Traded ratio</th>
                <td>{traded_ratio_before}</td>
                <td>{traded_ratio_after}</td>
            </tr>            
            <tr>
                <th>Traded Volume</th>
                <td>{traded_volume_before}</td>
                <td>{traded_volume_after}</td>
            </tr>
            <tr>
                <th>Traded cost</th>
                <td>{traded_cost_before}</td>
                <td>{traded_cost_after}</td>
            </tr> 
            <tr>
                <th>Traded price</th>
                <td>{traded_priceavg_before}</td>   
                <td>{traded_priceavg_after}</td>                    
            </tr>
            <tr>
                <td></td>
                <td></td>
                <td></td>
            </tr>
        </tbody>
    </table>
    """

In [ ]:
def print_summary_shaping(*, 
                          shaping_ratio_before, 
                          shaping_ratio_after, 
                          shaping_volume_before,
                          shaping_volume_after,
                          shaping_cost_before,
                          shaping_cost_after,
                          shaping_priceavg_before,
                          shaping_priceavg_after,
                         ):
    return f"""
    <table class="summary">
        <tbody>
            <tr>
                <td></td>
                <th>Actual</th>
                <th>Proposed</th>
            </tr>           
            <tr>
                <th>Shaping ratio</th>
                <td>{shaping_ratio_before}</td>
                <td>{shaping_ratio_after}</td>
            </tr>
            <tr>
                <th>Shaping volume</th>
                <td>{shaping_volume_before}</td>   
                <td>{shaping_volume_after}</td>                    
            </tr>
            <tr>
                <th>Shaping cost</th>
                <td>{shaping_cost_before}</td>
                <td>{shaping_cost_after}</td>   
            </tr>
            <tr>
                <th>Shaping price</th>
                <td>{shaping_priceavg_before}</td>
                <td>{shaping_priceavg_after}</td>   
            </tr>
        </tbody>
    </table>
    """

In [ ]:
def print_summary_hedges(*,
                         hedges_df: pd.DataFrame,
                         hedges_volume,
                         hedges_cost,
                         hedges_price,
                         ):
    
    if hedges_df.empty:
        return ""
    else:
        hedges=""
        for row in hedges_df.itertuples():
            hedges += f"""
                <tr>
                    <td colspan=2>
                        {'buy' if row.quantity>0 else 'sell'} {abs(row.quantity)} {row.calendar_delivery} {row.intraday_delivery} 
                    </td>
                </tr>
            """

        return f"""
        <table class="summary">
            <tbody>
                {hedges}
                <tr>
                    <th>Summary:</th>
                </tr>
                <tr>
                    <td>{hedges_volume:,.0f} MWh</td>
                </tr>  
                <tr>
                    <td>{hedges_cost:,.0f} €</td>
                </tr>            
                <tr>
                    <td>{hedges_price:,.2f} €/MWh</td>
                </tr>
            </tbody>
        </table>
        """

In [ ]:
def update_summary():
    global START
    global END
    temp_all_volumes = trades_volume_df.total_volume[START:END]
    temp_all_values = trades_volume_df.total_value[START:END]
    
    temp_conso_volumes = conso_df.consumption[START:END]
    temp_net_volumes = (conso_df.consumption - trades_volume_df.total_volume)[START:END]
    
    temp_market_curve = curve_pwrte_df.price[START:END]
    
    # Before
    traded_ratio_before = f"{temp_all_volumes.sum()/temp_conso_volumes.sum():,.0%}"
    traded_volume_before = f"{temp_all_volumes.sum():,.0f} MWh"
    traded_cost_before = f"{temp_all_values.sum():,.0f} €" 
    traded_priceavg_before = f"{temp_all_values.sum()/temp_all_volumes.sum():,.2f} €/MWh"
    
    shaping_ratio_before = f"{abs(temp_net_volumes).sum()/temp_conso_volumes.sum():,.0%}"
    shaping_volume_before = f"{abs(temp_net_volumes).sum():,.0f} MWh"
    shaping_cost_before = f"{(temp_net_volumes * temp_market_curve).sum():,.0f} €"
    shaping_priceavg_before = f"{(temp_net_volumes * temp_market_curve).sum() / temp_net_volumes.sum() :,.2f} €/MWh"

    # After
    if not HEDGES_VOLUMES.empty:
        temp_hedges_volumes = HEDGES_VOLUMES[START:END].reindex(temp_net_volumes.index).fillna(0)
        
        hedges_volume = temp_hedges_volumes.sum()
        hedges_cost = (temp_hedges_volumes * temp_market_curve).sum()
        hedges_price = hedges_cost/hedges_volume  
        
        traded_ratio_after = f"{(temp_all_volumes+temp_hedges_volumes).sum()/temp_conso_volumes.sum():,.0%}"
        traded_volume_after_value = (temp_all_volumes+temp_hedges_volumes).sum()
        traded_volume_after = f"{traded_volume_after_value:,.0f} MWh"
        traded_cost_after_value = temp_all_values.sum() + hedges_cost
        traded_cost_after = f"{traded_cost_after_value:,.0f} €"
        traded_priceavg_after = f"{traded_cost_after_value/traded_volume_after_value:,.2f} €/MWh"       
        
        shaping_ratio_after = f"{abs(temp_net_volumes-temp_hedges_volumes).sum()/temp_conso_volumes.sum():,.0%}"
        shaping_volume_after = f"{abs(temp_net_volumes-temp_hedges_volumes).sum():,.0f} MWh"
        shaping_cost_after = f"{((temp_net_volumes-temp_hedges_volumes) * temp_market_curve).sum():,.0f} €"
        shaping_priceavg_after = f"{((temp_net_volumes-temp_hedges_volumes) * temp_market_curve).sum() / (temp_net_volumes-temp_hedges_volumes).sum() :,.2f} €/MWh"
        
        

    else:
        traded_ratio_after = ""
        traded_volume_after = "" 
        traded_cost_after = ""
        traded_priceavg_after = ""        
        
        shaping_ratio_after = ""
        shaping_volume_after = ""
        shaping_cost_after = ""
        shaping_priceavg_after = ""
        
        hedges_volume = 0
        hedges_cost = 0
        hedges_price = 0
 
    # Updating summaries
    summary_trading_widget.value = nptt.NOOS_STYLE_TABLE_SUMMARY + print_summary_trading(
        traded_ratio_before=traded_ratio_before, 
        traded_ratio_after=traded_ratio_after, 
        traded_volume_before=traded_volume_before,
        traded_volume_after=traded_volume_after,
        traded_cost_before=traded_cost_before, 
        traded_cost_after=traded_cost_after, 
        traded_priceavg_before=traded_priceavg_before,
        traded_priceavg_after=traded_priceavg_after,
    )
    
    summary_shaping_widget.value = nptt.NOOS_STYLE_TABLE_SUMMARY + print_summary_shaping(
        shaping_ratio_before=shaping_ratio_before, 
        shaping_ratio_after=shaping_ratio_after, 
        shaping_volume_before=shaping_volume_before,
        shaping_volume_after=shaping_volume_after,
        shaping_cost_before=shaping_cost_before,
        shaping_cost_after=shaping_cost_after,
        shaping_priceavg_before=shaping_priceavg_before,
        shaping_priceavg_after=shaping_priceavg_after,
    )
    
    summary_hedges_widget.value = nptt.NOOS_STYLE_TABLE_SUMMARY + print_summary_hedges(
        hedges_df=HEDGES_DF,
        hedges_volume=hedges_volume,
        hedges_cost=hedges_cost,
        hedges_price=hedges_price,
    )

### Trading

In [ ]:
summary_trading_widget

### Shaping

In [ ]:
summary_shaping_widget

# Sidebar

#### wholesale

In [ ]:
MISSING_CALENDAR_DELIVERY = "-"

In [ ]:
indices_df = npta.domain.products.collections.IndexSet.from_pandas(products_df).to_pandas()

products_values = indices_df.calendar_delivery.unique()


products_base_selection = ipywidgets.SelectMultiple(
    options=indices_df[indices_df.intraday_delivery=='BASE'].calendar_delivery,
    value=(),
    rows=len(products_values),
    disabled=False
)

products_peak_selection = ipywidgets.SelectMultiple(
    options=[i if (i in list(indices_df[indices_df.intraday_delivery=='PEAK'].calendar_delivery)) else MISSING_CALENDAR_DELIVERY
             for i in indices_df[indices_df.intraday_delivery=='BASE'].calendar_delivery ],
    value=(),
    rows=len(products_values),
    disabled=False
)

products_base_vbox = ipywidgets.VBox([ipywidgets.Label('BASE'), products_base_selection])
products_peak_vbox = ipywidgets.VBox([ipywidgets.Label('PEAK'), products_peak_selection])

products_hbox = ipywidgets.HBox([products_base_vbox, products_peak_vbox])

In [ ]:
def get_products_list():
    df=pd.DataFrame()
    df["calendar_delivery"] = products_base_selection.value + products_peak_selection.value
    df["intraday_delivery"] = ('BASE',) * len(products_base_selection.value) + ('PEAK',) * len(products_peak_selection.value)
    df["asset"] = "PWRTE"
    df["quantity"] = 1
    return df[df.calendar_delivery!=MISSING_CALENDAR_DELIVERY]

#### Interactivity

In [ ]:
def update_hedges():
    global HEDGES_DF
    global HEDGES_VOLUMES
    select_products_df = get_products_list()
    net_volumes = conso_df.consumption - trades_volume_df.total_volume
    if not select_products_df.empty:
        select_products_volumes = npta.get_volumes_from_hedges(select_products_df)
        HEDGES_DF = npta.get_hedges_from_volumes(net_volumes.reindex(select_products_volumes.index).fillna(0), products=select_products_df).astype({'quantity': 'int'})
        HEDGES_DF = HEDGES_DF[HEDGES_DF.quantity!=0]
        HEDGES_VOLUMES = npta.get_volumes_from_hedges(HEDGES_DF)
        update_graph_hedges()
    else:
        HEDGES_DF = pd.DataFrame()
        HEDGES_VOLUMES = pd.DataFrame()

In [ ]:
def update_products(change):
    update_hedges()
    update_summary()

def update_range(layout, _):
    global START
    global END
    xaxis_range = layout.xaxis.range
    
    # range mask
    START =  dt.datetime.fromisoformat(xaxis_range[0][:10]).astimezone(pytz.UTC)
    END =  dt.datetime.fromisoformat(xaxis_range[1][:10]).astimezone(pytz.UTC)

    update_summary()


In [ ]:
products_base_selection.observe(update_products, 'value')
products_peak_selection.observe(update_products, 'value')

fig.layout.on_change(update_range, ('xaxis', 'range'))

In [ ]:
def init_range():
    global START
    global END
    if fig.layout.xaxis.range is None:
        xaxis_range = (fig.data[1].x[0].isoformat(), fig.data[1].x[-1].isoformat())
    else:
        xaxis_range = fig.layout.xaxis.range
        
    # range mask
    START =  dt.datetime.fromisoformat(xaxis_range[0][:10]).astimezone(pytz.UTC)
    END =  dt.datetime.fromisoformat(xaxis_range[1][:10]).astimezone(pytz.UTC)

def init():
    update_hedges()
    init_range()
    
    
init()

### Select instruments

In [ ]:
products_hbox

### Select strategy

In [ ]:
ipywidgets.ToggleButtons(
    options=['volume', 'value'],
    disabled=True,
    tooltips=['hedge to 100% traded ratio', 'hedge to zero shaping cost and minimum shaping volume'],
)

### Proposition

In [ ]:
summary_hedges_widget